In [ ]:
####AUTOENCODER
optimizer = tf.keras.optimizers.Adam(0.0001)
latent_dim = 128
original_dim = (sr16k, 1, 1)
encoder_inputs = keras.Input(original_dim)                                                   # bat, samps, ?, ch
x = layers.Conv2D(64, (33,1), activation="relu", padding="same",strides=(4,1))(encoder_inputs) # [69, 48000, 1, 1] -> [69, 12000, 1, 64]
x = layers.Conv2D(128,(33,1), activation="relu", padding="same",strides=(4,1))(x)
x = layers.Conv2D(256,(33,1), activation="relu", padding="same",strides=(4,1))(x)
x = layers.Conv2D(512,(33,1), activation="relu", padding="same",strides=(3,1))(x)
encoder_flat = layers.Flatten()(x)
latent = layers.Dense(latent_dim, activation='relu')(encoder_flat)

encoder = keras.Model(encoder_inputs, latent, name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(latent_dim)
x = layers.Dense(250 * 1 * 512, activation="relu")(latent_inputs)
x = layers.Reshape((250, 1, 512))(x)
x = layers.Conv2DTranspose(128,(33,1), activation="relu",padding= "same",strides= (4,1))(x)
x = layers.Conv2DTranspose(64,(33,1), activation="relu",padding= "same",strides= (4,1))(x)
decoder_outputs = layers.Conv2DTranspose(1, (33, 1), activation="tanh", padding="same", strides = (4,1))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="AE")

decoder.summary()

In [ ]:
salida = decoder(encoder(encoder_inputs)) # note, outputs available from encoder model are z_mean, z_log_sigma and z. We take z by specifying [2]
# Instantiate a VAE model
ae = Model(inputs=encoder_inputs, outputs= salida, name='AE-Model')
ae.compile(optimizer= optimizer,loss = 'mean_squared_error')


In [ ]:
tr_logs = ae.fit(Xw,Xw, shuffle = True, batch_size= 5, epochs= 100)

In [ ]:
#######LOSS
plt.plot(tr_logs.history['loss'])
#plt.plot(tr_logs.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
ae_latent = encoder.predict(Xw)


In [ ]:
###visualización PCA
fig = plt.figure(1, figsize=(12, 6))
ax = Axes3D(fig)#,elev=-0, azim= 0)
#X_reduced = PCA(n_components=3).fit_transform(X)

# create the PCA instance with the number of components to be computed 
pca = PCA(n_components= 3)

# "transform" the data that you want to reduce to "two components"
X_PCA = pca.fit_transform(ae_latent)

ax.scatter3D(X_PCA[:68, 0], X_PCA[:68, 1],X_PCA[:68, 2], c='red', label='BluesDriver')
ax.scatter3D(X_PCA[69:137, 0], X_PCA[69:137, 1],X_PCA[69:137, 2], c='green', label='Chorus')
ax.scatter3D(X_PCA[138:206, 0], X_PCA[138:206, 1],X_PCA[138:206, 2], c='blue', label='Clean')
ax.scatter3D(X_PCA[207:275, 0], X_PCA[207:275, 1],X_PCA[207:275, 2], c='red', label='Digital Delay')
ax.scatter3D(X_PCA[276:344, 0], X_PCA[276:344, 1],X_PCA[276:344, 2], c='yellow', label= 'Flanger')
ax.scatter3D(X_PCA[345:413, 0], X_PCA[345:413, 1],X_PCA[345:413, 2], c='black', label='Phaser')
ax.scatter3D(X_PCA[414:482, 0], X_PCA[414:482, 1],X_PCA[414:482, 2], c='white', label='RAT')
ax.scatter3D(X_PCA[483:551, 0], X_PCA[483:551, 1],X_PCA[483:551, 2], c='orange', label='Sweep Echo')
ax.scatter3D(X_PCA[552:620, 0], X_PCA[552:620, 1],X_PCA[552:620, 2], c='pink', label='TapeEcho')
ax.scatter3D(X_PCA[621:689, 0], X_PCA[621:689, 1],X_PCA[621:689, 2], c='brown', label='TubeScreamer')

plt.legend()
plt.title('Analisis de componentes Principales "2D')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()

In [ ]:
fig = plt.figure(1, figsize=(12, 6))
# create the PCA instance with the number of components to be computed 
pca = PCA(n_components= 2)

# "transform" the data that you want to reduce to "two components"
X_PCA = pca.fit_transform(ae_latent)

plt.scatter(X_PCA[:68, 0], X_PCA[:68, 1], c='red', label='BluesDriver')
plt.scatter(X_PCA[69:137, 0], X_PCA[69:137, 1], c='green', label='Chorus')
plt.scatter(X_PCA[138:206, 0], X_PCA[138:206, 1], c='blue', label='Clean')
plt.scatter(X_PCA[207:275, 0], X_PCA[207:275, 1], c='red', label= 'Digital Delay')
plt.scatter(X_PCA[276:344, 0], X_PCA[276:344, 1], c='yellow', label= 'Flanger')
plt.scatter(X_PCA[345:413, 0], X_PCA[345:413, 1], c='black', label= 'Phaser')
plt.scatter(X_PCA[414:482, 0], X_PCA[414:482, 1], c='white', label= 'RAT')
plt.scatter(X_PCA[483:551, 0], X_PCA[483:551, 1], c='orange', label= 'Sweep Echo')
plt.scatter(X_PCA[552:620, 0], X_PCA[552:620, 1], c='pink', label= 'TapeEcho')
plt.scatter(X_PCA[621:689, 0], X_PCA[621:689, 1], c='brown', label= 'TubeScreamer')

plt.legend()
plt.title('Analisis de componentes Principales "2D')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()

In [ ]:
###visualización PCA 3D #########
###visualización PCA 3D #########
###visualización PCA 3D #########

fig = plt.figure(1, figsize=(12, 6))
ax = Axes3D(fig,elev=-0, azim=0)
pca = PCA(n_components= 3)
X_PCA = pca.fit_transform(ae_latent)

ax.scatter3D(X_PCA[:689, 0], X_PCA[:689, 1],X_PCA[:689, 2], c='red', label='BluesDriver')
ax.scatter3D(X_PCA[690:1379, 0], X_PCA[690:1379, 1],X_PCA[690:1379, 2], c='green', label='Chorus')
ax.scatter3D(X_PCA[1380:2069, 0], X_PCA[1380:2069, 1],X_PCA[1380:2069, 2], c='blue', label='Clean')
ax.scatter3D(X_PCA[2070:2759, 0], X_PCA[2070:2759, 1],X_PCA[2070:2759, 2], c='cyan', label='Digital Delay')
ax.scatter3D(X_PCA[2760:3449, 0], X_PCA[2760:3449, 1],X_PCA[2760:3449, 2], c='yellow', label='Flanger')
ax.scatter3D(X_PCA[3450:4139, 0], X_PCA[3450:4139, 1],X_PCA[3450:4139, 2], c='black', label='Phaser')
ax.scatter3D(X_PCA[4140:4829, 0], X_PCA[4140:4829, 1],X_PCA[4140:4829, 2], c='purple', label='RAT')
ax.scatter3D(X_PCA[4830:5519, 0], X_PCA[4830:5519, 1],X_PCA[4830:5519, 2], c='orange', label= 'Sweep Echo')
ax.scatter3D(X_PCA[5520:6209, 0], X_PCA[5520:6209, 1],X_PCA[5520:6209, 2], c='pink', label='TapeEcho')
ax.scatter3D(X_PCA[6210:6899, 0], X_PCA[6210:6899, 1],X_PCA[6210:6899, 2], c='brown', label='TubeScreamer')

plt.legend()
plt.title('Analisis de componentes Principales "3D')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
#plt.zlabel('PCA 3')

plt.show()

In [ ]:
#####PCA 2D ##########
#####PCA 2D ##########
#####PCA 2D ##########

fig = plt.figure(1, figsize=(12, 6))
pca = PCA(n_components= 2)
X_PCA = pca.fit_transform(ae_latent)

plt.scatter(X_PCA[:689, 0], X_PCA[:689, 1], c='red', label= 'BluesDriver')
plt.scatter(X_PCA[690:1379, 0], X_PCA[690:1379, 1], c='green', label= 'Chorus')
plt.scatter(X_PCA[1380:2069, 0], X_PCA[1380:2069, 1], c='blue', label= 'Clean')
plt.scatter(X_PCA[2070:2759, 0], X_PCA[2070:2759, 1], c='cyan', label= 'Digital Delay')
plt.scatter(X_PCA[2760:3449, 0], X_PCA[2760:3449, 1], c='yellow', label= 'Flanger')
plt.scatter(X_PCA[3450:4139, 0], X_PCA[3450:4139, 1], c='black', label= 'Phaser')
plt.scatter(X_PCA[4140:4829, 0], X_PCA[4140:4829, 1], c='purple', label= 'RAT')
plt.scatter(X_PCA[4830:5519, 0], X_PCA[4830:5519, 1], c='orange', label= 'Sweep Echo')
plt.scatter(X_PCA[5520:6209, 0], X_PCA[5520:6209, 1], c='pink', label= 'TapeEcho')
plt.scatter(X_PCA[6210:6899, 0], X_PCA[6210:6899, 1], c='brown', label= 'TubeScreamer')

plt.legend()
plt.title('Analisis de componentes Principales AUTOENCODER')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()